In [1]:
# DB 접속 정보 불러오기
with open('c:/2nd_project/Service/DB_config.txt') as f:
    conf = f.readlines()
    DB_HOST = conf[0].replace('\n','')
    DB_USER = conf[1].replace('\n','')
    DB_PASSWORD = conf[2].replace('\n','')
    DB_NAME = conf[3].replace('\n','')

In [2]:
print(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME)

127.0.0.1 root 0000 project_chatbot


In [3]:
import pymysql
import pymysql.cursors
import logging

class Database:
    def __init__(self, host, user, password, db_name, charset='utf8'):
        self.host = host
        self.user = user
        self.password = password
        self.db_name = db_name
        self.charset = charset
        self.conn = None

    def connect(self):
        if self.conn != None:
            return
        self.conn = pymysql.connect(
            host=self.host,
            user=self.user,
            password=self.password,
            db=self.db_name,
            charset=self.charset
        )

    def close(self):
        if self.conn is None:
            return
        if not self.conn.open:
            self.conn = None
            return
        self.conn.close()
        self.conn = None

    def execute(self, sql):
        last_row_id = -1
        try:
            with self.conn.cursor() as cursor:
                cursor.execute(sql)
            self.conn.commit()
            last_row_id = cursor.lastrowid
        except Exception as ex:
            logging.error(ex)
        finally:
            return last_row_id
        
    def select_one(self, sql):
        result = None
        try:
            with self.conn.cursor(pymysql.cursors.DictCursor) as cursor:
                cursor.execute(sql)
                result = cursor.fetchone()
        except Exception as ex:
            logging.error(ex)
        finally:
            return result

    def select_all(self, sql):
        result = None
        try:
            with self.conn.cursor(pymysql.cursors.DictCursor) as cursor:
                cursor.execute(sql)
                result = cursor.fetchall()
        except Exception as ex:
            logging.error(ex)
        finally:
            return result

In [4]:
import pandas as pd

df = pd.read_excel('c:/2nd_project/Data/movie_data/[KOBIS] 박스오피스_줄거리_감정키워드(2003.01~2023.07).xlsx', engine='openpyxl')
df.iloc[0]

영화명                                                  #살아있다
개봉일                                             2020-06-24
누적관객수                                              1903992
등급                                                15세이상관람가
장르                                                     드라마
대표국적                                                    한국
국적                                                      한국
제작사                                      영화사 집,(주)퍼스펙티브픽쳐스
배급사                                      롯데컬처웍스(주)롯데엔터테인먼트
감독                                                     조일형
배우       유아인,박신혜,전배수,고나영,권용채,김경태,김다영,김단비,김라희,김미래,김미래,김윤...
줄거리                                                    NaN
키워드                                                    NaN
Name: 0, dtype: object

In [5]:
# title, opendate, people, grade, genre, repnation, nations, Production, distributor, director, actors, story, keyword 

In [6]:
# insert 쿼리 생성
def InsertMovie(series):
    try: 
        sql = f'''insert chat_movie(title, opendate, people, grade, genre, repnation, nations, 
        Production, distributor, director, actors, story, keyword) values ("{series[0]}", "{series[1]}",
        "{series[2]}", "{series[3]}", "{series[4]}", "{series[5]}", "{series[6]}", "{series[7]}", "{series[8]}",
        "{series[9]}", "{series[10]}", "{series[11]}", "{series[12]}")'''

        # 엑셀에서 불러온 cell에 데이터가 없는 경우 null로 치환
        sql = sql.replace('nan', 'null').replace('None','null')
        
        db.execute(sql)
        
    except Exception as ex:
        logging.error(ex)

In [7]:
db = Database(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, db_name=DB_NAME)
db.connect() 

for i in range(len(df)):
    InsertMovie(df.iloc[i])

# 답변 찾기

In [4]:
class FindAnswer:
    def __init__(self, db):
        self.db = db
        
    # 검색 쿼리 생성
    def _make_query(self, lst):
        self.intent = lst[0]
        self.situation = lst[1]
        self.emotion = lst[2]
        self.story = lst[3]
        
        sql = 'select * from chat_movie'
        if self.intent == "추천":
            if self.story != "-":
                sql = sql + f" where keyword='{self.story}'"

        # 누적관객수 100만명 이상이고 답변이 여러개일 경우 랜덤으로 하나만 뽑기
        sql = sql + " and people>=1000000 order by rand() limit 1"
        return sql 

In [9]:
# db = Database(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, db_name=DB_NAME)
# db.connect() 

# # sql = "select * from chat_movie where keyword='학교' and people>=1000000 order by rand() limit 1"
# # sql = "select * from chat_movie where story like '%가족%' and people>=1000000 order by rand() limit 1"
# # sql = "select * from chat_movie where story like '%눈물%' and people>=1000000 order by rand() limit 1"

# db.select_all(sql)


In [5]:
sql = 'select * from chat_movie'
lst = ['추천','슬픔','부정','인기']

if lst[0] == "추천":
    if lst[2] == "긍정":
        sql_q = " keyword like "
    elif lst[2] == '부정':
        sql_q = " keyword not like "
    else:
        sql_q = " keyword not like "
    
    if lst[1] != "없음":
        sql = sql + " where" + sql_q + f"'%{lst[1]}%'" +" and keyword !='null' and" 
    else:
        sql = sql + " where"
        
    if lst[3] == '최신':
        sql = sql + " people>=1000000 order by opendate DESC limit 1"
    elif lst[3] == '인기':
        sql = sql + " people>=5000000 order by rand() limit 1"
    else:
        sql = sql + " people>=1000000 order by rand() limit 1"
sql

"select * from chat_movie where keyword not like '%슬픔%' and keyword !='null' and people>=5000000 order by rand() limit 1"

In [6]:
db = Database(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, db_name=DB_NAME)
db.connect() 

db.select_all(sql)

[{'title': '알라딘',
  'opendate': '2019-05-23',
  'people': '12797927',
  'grade': '전체관람가',
  'genre': '어드벤처,가족,판타지',
  'repnation': '미국',
  'nations': '미국',
  'Production': 'null',
  'distributor': '월트디즈니컴퍼니코리아 유한책임회사',
  'director': '가이 리치',
  'actors': '메나 마수드,윌 스미스,나오미 스콧',
  'story': '머나먼 사막 속 신비의 아그라바 왕국의 시대.좀도둑 ‘알라딘’은 마법사 ‘자파’의 의뢰로 마법 램프를 찾아 나섰다가 주인에게 세 가지 소원을 들어주는 지니를 만나게 되고, 자스민 공주의 마음을 얻으려다 생각도 못했던 모험에 휘말리게 되는데…',
  'keyword': '신남'}]

# 모델 테스트

In [7]:
from konlpy.tag import Komoran
import pickle
import jpype

class Preprocess :
    def __init__(self, word2index_dic='', userdic=None):
        # 단어 인덱스 사전 불러오기
        if (word2index_dic != ''):
            f = open(word2index_dic, 'rb')
            self.word_index = pickle.load(f)
            f.close()
        else:
            self.word_index = None
            
        # 형태소 분석기 초기화
        self.komoran = Komoran(userdic=userdic)
        
        # 제외할 품사
        # 참조 : https://docs.komoran.kr/firststep/postypes.html
        # 관계언, 기호, 어미, 접미사 제거
        self.exclusion_tags = [
            'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
            'JX', 'JC',
            'SF', 'SP', 'SS', 'SE', 'SO',
            'EP', 'EF', 'EC', 'ETN', 'ETM',
            'XSN', 'XSV', 'XSA'
        ]
        
    # 형태소 분석기
    def pos(self, sentence):
        jpype.attachThreadToJVM()
        return self.komoran.pos(sentence)
        
    # 불용어 제거 후, 필요한 품사 정보만 가져오기
    def get_keywords(self, pos, without_tag=False):
        f = lambda x: x in self.exclusion_tags
        word_list = []
        for p in pos:
            if f(p[1]) is False:
                word_list.append(p if without_tag is False else p[0])
        return word_list
    
    # 키워드를 단어 인덱스 시퀀스로 변환
    def get_wordidx_sequence(self, keywords):
        if self.word_index is None:
            return []
        
        w2i = []
        for word in keywords:
            try:
                w2i.append(self.word_index[word])
            except KeyError:
                # 해당 단어가 사전에 없는 경우, OOV 처리
                w2i.append(self.word_index['OOV'])
        return w2i

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing
import numpy as np

# 분류 모델 모듈
class PredictModel:
    def __init__(self, category, model_name, proprocess):
        
        self.MAX_SEQ_LEN = 15
        self.category = category
        
        if self.category =='intent':
            self.labels = {0: '기타', 1: '추천', 2: '후기', 3: '정보', 4: '예매', 5: '욕설'}
            
        elif self.category == 'emotion':
            self.labels = {0: '무서움', 1: '슬픔', 2: '신남', 3: '없음', 4: '웃김', 5: '재미'}
            
        elif self.category == 'binary':
            self.labels = {0: '긍정', 1: '부정', 2: '없음'}
            
        elif self.category == 'trend':
            self.labels = {0: '없음', 1: '인기', 2: '최신'}
            
        elif self.category == 'ner':
            self.MAX_SEQ_LEN = 40
            self.labels = {1: 'O', 2: 'B_MOVIE', 3: 'B_ACT', 4: 'B_GEN', 5: 'B_NAT', 6: 'B_DIR', 7: 'B_DT', 8: 'B_RAT', 0: 'PAD'}
            
        else:
            self.labels = {}

        self.labels[len(self.labels)]="-"
        # 분류 모델 불러오기
        self.model = load_model(model_name)
        # 챗봇 Preprocess 객체
        self.p = proprocess

    # 클래스 예측
    def predict_class(self, query):
        # 형태소 분석
        pos = self.p.pos(query)
        
        # 문장내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 패딩처리
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=self.MAX_SEQ_LEN, padding='post')
        predict = self.model.predict(padded_seqs)
        predict_class = tf.math.argmax(predict, axis=1)

        return predict_class.numpy()[0]

    # ner_전용
    def predict_ner(self, query):
        # 형태소 분석
        pos = self.p.pos(query)

        # 문장내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 패딩처리
        max_len = 40
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, padding="post", value=0,
                                                           maxlen=max_len)
        predict = self.model.predict(np.array([padded_seqs[0]]))
        predict_class = tf.math.argmax(predict, axis=-1)
        tags = [self.labels[i] for i in predict_class.numpy()[0]]
        return list(zip(keywords, tags))

In [56]:
# GPU에서 실행
import tensorflow as tf
with tf.device('/:GPU0'):
        p = Preprocess(word2index_dic='c:/2nd_project/Data/chatbot_dict.bin',
                        userdic = 'c:/2nd_project/Data/ner_data/new_user_dic_10.txt')

        intent = PredictModel(category='intent', model_name='c:/2nd_project/Model/intent_model/intent_uso_model_0811_a_ep_best(5).h5', proprocess=p)
        emotion = PredictModel(category='emotion', model_name='c:/2nd_project/Model/emotion_model/question_emotion_model.h5', proprocess=p)
        binary = PredictModel(category='binary', model_name='c:/2nd_project/Model/binary_model/question_emotion_binary_model.h5', proprocess=p)
        trend = PredictModel(category='trend', model_name='c:/2nd_project/Model/trend_model/question_trend_model.h5', proprocess=p)
        ner = PredictModel(category='ner', model_name='c:/2nd_project/Model/ner_model/ner_model_0817_a.h5', proprocess=p)

        def ner_tag_sep(lsts):
                ner_movie = []
                ner_act = []
                ner_gen = []
                ner_nat = []
                ner_dir = []
                ner_dt = []
                ner_rat = []

                for lst in lsts:
                        if lst[1] == 'B_MOVIE':
                                ner_movie.append(lst[0])
                        elif lst[1] == 'B_ACT':
                                ner_act.append(lst[0])
                        elif lst[1] == 'B_GEN':
                                ner_gen.append(lst[0])
                        elif lst[1] == 'B_NAT':
                                ner_nat.append(lst[0])
                        elif lst[1] == 'B_DIR':
                                ner_dir.append(lst[0])
                        elif lst[1] == 'B_DT':
                                ner_dt.append(lst[0])
                        elif lst[1] == 'B_RAT':
                                ner_rat.append(lst[0])

                        ner_movie = list(set(ner_movie))
                        ner_act = list(set(ner_act))
                        ner_gen = list(set(ner_gen))
                        ner_nat = list(set(ner_nat))
                        ner_dir = list(set(ner_dir))
                        ner_dt = list(set(ner_dt))
                        ner_rat = list(set(ner_rat))

                return ner_movie, ner_act, ner_gen, ner_nat, ner_dir, ner_dt, ner_rat

        def predict_keyword(text):
        
                intent_pred = intent.predict_class(text)
                emotion_pred = emotion.predict_class(text)
                binary_pred = binary.predict_class(text)
                trend_pred = trend.predict_class(text)
                ner_pred = ner.predict_ner(text)
                
                intent_label = intent.labels[intent_pred]
                emotion_label = emotion.labels[emotion_pred]
                binary_label = binary.labels[binary_pred]
                trend_label = trend.labels[trend_pred]
                ner_label = ner_tag_sep(ner_pred)

                return intent_label, emotion_label, binary_label, trend_label, ner_label

In [57]:
# # intent = lsts[0]
# # emotion = lsts[1] / 6
# # binary = lsts[2]  / 3
# # trend = lsts[3]   / 3
# # movie = lsts[4][0]
# # actor = lsts[4][1]
# # genre = lsts[4][2]
# # nation = lsts[4][3]
# # director = lsts[4][4]
# # date = lsts[4][5]
# # rating = lsts[4][6]

# # 아무 영화 추천(lsts[0]으로만)
# # (감정) 영화 추천(lsts[1], lsts[2])
# # (트랜드) 영화 추천(lsts[3])
# # 영화이름으로 추천 / 배우명으로 추천 / 장르별 추천 / 국가별 추천 / 감독명으로 추천 / 등급별 추천

# # ('추천', '슬픔', '긍정', '인기', ([], ['박서준'], [], [], ['봉준호'], [], []))

#         # if self.category =='intent':
#         #     self.labels = {0: '기타', 1: '추천', 2: '후기', 3: '정보', 4: '예매', 5: '욕설'}
            
#         # elif self.category == 'emotion':
#         #     self.labels = {0: '무서움', 1: '슬픔', 2: '신남', 3: '없음', 4: '웃김', 5: '재미'}
            
#         # elif self.category == 'binary':
#         #     self.labels = {0: '긍정', 1: '부정', 2: '없음'}
            
#         # elif self.category == 'trend':
#         #     self.labels = {0: '없음', 1: '인기', 2: '최신'}
            
#         # elif self.category == 'ner':
#         #     self.MAX_SEQ_LEN = 40
#         #     self.labels = {1: 'O', 2: 'B_MOVIE', 3: 'B_ACT', 4: 'B_GEN', 5: 'B_NAT', 6: 'B_DIR', 7: 'B_DT', 8: 'B_RAT', 0: 'PAD'}

# def sql_querty(lsts):
#     sql = 'select * from chat_movie'

#     if lsts[0] == "추천":
#         if lsts[2] == "긍정":
#             sql_q = " keyword like "
#         elif lsts[2] == '부정':
#             sql_q = " keyword not like "
#         else:
#             sql_q = " keyword not like "
        
#         if lsts[1] != "없음":
#             sql = sql + " where" + sql_q + f"'%{lst[1]}%'" +" and keyword !='null' and" 
#         else:
#             sql = sql + " where"
            
#         if lsts[3] == '최신':
#             sql = sql + " people>=1000000 order by opendate DESC limit 1"
#         elif lsts[3] == '인기':
#             sql = sql + " people>=5000000 order by rand() limit 1"
#         else:
#             sql = sql + " people>=1000000 order by rand() limit 1"
    
#     return sql

In [50]:
# # text = '기생충 영화 드림 기생충 감독 봉준호 내가 좋아하는 배우는 황정민 박서준 고소영 우리나라 호러 장르는 액션  미국 오늘은 8월 전체관람가인 영화 추천 12시 안해주면 화날꺼같아'
# text = '천만 넘은 봉준호 감독에 박서준 나온 재밌는 영화 없어서 슬픈데 영화 추천좀'

# lsts = predict_keyword(text)

# print('1. 의도 :', lsts[0])
# print('2. 감정 :', lsts[1])
# print('3. 긍부정 :', lsts[2])
# print('4. 트렌드 :', lsts[3])
# print('5. 개체명\n\t영화명 : {}\n\t배우 : {}\n\t장르 : {}\n\t국가 : {}\n\t감독 : {}\n\t시간 : {}\n\t등급 : {}'
# .format(lsts[4][0], lsts[4][1], lsts[4][2], lsts[4][3], lsts[4][4], lsts[4][5], lsts[4][6]))

# # sql = sql_querty(lst)
# # print(sql)
# # db.select_all(sql)

1/1 [==============================] - 0s 129ms/step
1. 의도 : 추천
2. 감정 : 슬픔
3. 긍부정 : 긍정
4. 트렌드 : 인기
5. 개체명
	영화명 : []
	배우 : ['박서준']
	장르 : []
	국가 : []
	감독 : ['봉준호']
	시간 : []
	등급 : []


In [ ]:
# intent = lsts[0]
# emotion = lsts[1] / 6
# binary = lsts[2]  / 3
# trend = lsts[3]   / 3
# movie = lsts[4][0]
# actor = lsts[4][1]
# genre = lsts[4][2]
# nation = lsts[4][3]
# director = lsts[4][4]
# date = lsts[4][5]
# rating = lsts[4][6]

# 아무 영화 추천(lsts[0]으로만)
# (감정) 영화 추천(lsts[1], lsts[2])
# (트랜드) 영화 추천(lsts[3])
# 영화이름으로 추천 / 배우명으로 추천 / 장르별 추천 / 국가별 추천 / 감독명으로 추천 / 등급별 추천

# ('추천', '슬픔', '긍정', '인기', ([], ['박서준'], [], [], ['봉준호'], [], []))

        # if self.category =='intent':
        #     self.labels = {0: '기타', 1: '추천', 2: '후기', 3: '정보', 4: '예매', 5: '욕설'}
            
        # elif self.category == 'emotion':
        #     self.labels = {0: '무서움', 1: '슬픔', 2: '신남', 3: '없음', 4: '웃김', 5: '재미'}
            
        # elif self.category == 'binary':
        #     self.labels = {0: '긍정', 1: '부정', 2: '없음'}
            
        # elif self.category == 'trend':
        #     self.labels = {0: '없음', 1: '인기', 2: '최신'}
            
        # elif self.category == 'ner':
        #     self.MAX_SEQ_LEN = 40
        #     self.labels = {1: 'O', 2: 'B_MOVIE', 3: 'B_ACT', 4: 'B_GEN', 5: 'B_NAT', 6: 'B_DIR', 7: 'B_DT', 8: 'B_RAT', 0: 'PAD'}

def sql_querty(lsts):
    sql = 'select * from chat_movie'

    if lsts[0] == "추천":
        if lsts[2] == "긍정":
            sql_q = " keyword like "
        elif lsts[2] == '부정':
            sql_q = " keyword not like "
        else:
            sql_q = " keyword not like "
        
        if lsts[1] != "없음":
            sql = sql + " where" + sql_q + f"'%{lst[1]}%'" +" and keyword !='null' and" 
        else:
            sql = sql + " where"
            
        if lsts[3] == '최신':
            sql = sql + " people>=1000000 order by opendate DESC limit 1"
        elif lsts[3] == '인기':
            sql = sql + " people>=5000000 order by rand() limit 1"
        else:
            sql = sql + " people>=1000000 order by rand() limit 1"
    
    return sql

# 답변

In [ ]:
# intent = lsts[0]
# emotion = lsts[1]
# binary = lsts[2]
# trend = lsts[3]
# movie = lsts[4][0]
# actor = lsts[4][1]
# genre = lsts[4][2]
# nation = lsts[4][3]
# director = lsts[4][4]
# date = lsts[4][5]
# rating = lsts[4][6]

In [70]:
# text = '기생충 영화 드림 기생충 감독 봉준호 내가 좋아하는 배우는 황정민 박서준 고소영 우리나라 호러 장르는 액션  미국 오늘은 8월 전체관람가인 영화 추천 12시 안해주면 화날꺼같아'
query = '천만 넘은 봉준호 감독에 박서준 나온 재밌는 영화 없어서 슬픈데 영화 추천좀'

lsts = predict_keyword(query)

print('1. 의도 :', lsts[0])
print('2. 감정 :', lsts[1])
print('3. 긍부정 :', lsts[2])
print('4. 트렌드 :', lsts[3])
print('5. 개체명\n\t영화명 : {}\n\t배우 : {}\n\t장르 : {}\n\t국가 : {}\n\t감독 : {}\n\t시간 : {}\n\t등급 : {}'
.format(lsts[4][0], lsts[4][1], lsts[4][2], lsts[4][3], lsts[4][4], lsts[4][5], lsts[4][6]))

print(lsts)

sql = sql_querty(lsts)
print(sql)

movie_data = db.select_all(sql)
print(movie_data)

recomand_movie = movie_data[0]['title']
print(recomand_movie)

1/1 [==============================] - 0s 197ms/step
1. 의도 : 추천
2. 감정 : 슬픔
3. 긍부정 : 긍정
4. 트렌드 : 인기
5. 개체명
	영화명 : []
	배우 : ['박서준']
	장르 : []
	국가 : []
	감독 : ['봉준호']
	시간 : []
	등급 : []
('추천', '슬픔', '긍정', '인기', ([], ['박서준'], [], [], ['봉준호'], [], []))
select * from chat_movie where keyword like '%슬픔%' and keyword !='null' and people>=5000000 order by rand() limit 1
[{'title': '신과함께-죄와 벌', 'opendate': '2017-12-20', 'people': '14414658', 'grade': '12세이상관람가', 'genre': '판타지,드라마', 'repnation': '한국', 'nations': '한국', 'Production': '리얼라이즈픽쳐스(주),(주)덱스터스튜디오', 'distributor': '롯데쇼핑㈜롯데엔터테인먼트', 'director': '김용화', 'actors': '하정우,차태현,주지훈,김향기,마동석,김동욱,도경수,오달수,임원희,장광,정해균,김수안,이정재,김해숙,이경영,김하늘,최용훈,강소백,정수윤,이수봉,이승준,이지향,김태준,정지훈,예수정,김기태,성유빈,오희준,이준혁,유수빈,윤지온,임철수,나철,강길우,김민종,홍인,정순원,김그림,박윤호,양지수,김지나,윤승훈', 'story': '저승 법에 의하면, 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다.살인, 나태, 거짓, 불의, 배신, 폭력, 천륜7개의 지옥에서 7번의 재판을 무사히 통과한 망자만이 환생하여 새로운 삶을 시작할 수 있다. “김자홍 씨께선, 오늘 예정 대로 무사히 사망하셨습니다”화재 사고 현장에서 여자아이를 구하고 죽음을 맞이한 소방관 자홍, 그의 앞에 저승차사 해원맥과 덕춘이 나

In [64]:
# 의도
answer_i = ''
if lsts[0] == '기타':
    answer_i = '의도를 잘 모르겠어요.'
elif lsts[0] == '추천':
    answer_i = '을(를) 추천드려요. :)'
elif lsts[0] == '후기':
    answer_i = '은(는) 후기가 있습니다.'
elif lsts[0] == '정보':
    answer_i = '은(는) 정보가 있습니다.'
elif lsts[0] == '예매':
    answer_i = '예매 되었습니다!'
elif lsts[0] == '욕설':
    answer_i = '욕설은 나빠요 :('
else:
    answer_i = 'intent error'

# 감정
answer = ''
if lsts[1] == '무서움':
    if lsts[2] == '긍정':
        answer_e = '무서운 영화인'
    else:
        answer_e = '무섭지 않은 영화인'
elif lsts[1] == '슬픔':
    if lsts[2] == '긍정':
        answer_e = '슬픈 영화인'
    else:
        answer_e = '슬프지 않은 영화인'
elif lsts[1] == '신남':
    if lsts[2] == '긍정':
        answer_e = '신나는 영화인'
    else:
        answer_e = '신나지 않은 영화인'
elif lsts[1] == '없음':
    answer_e = ''
elif lsts[1] == '웃김':
    if lsts[2] == '긍정':
        answer_e = '웃긴 영화인'
    else:
        answer_e = '웃기지 않은 영화인'
elif lsts[1] == '재미':
    if lsts[2] == '긍정':
        answer_e = '재미있는 영화인'
    else:
        answer_e = '재미없는 영화인'
else:
    answer_e = 'emotion error'

# 트렌드
if lsts[3] == '없음':
    answer_t = ''
elif lsts[3] == '인기':
    answer_t = '인기 있는'
elif lsts[3] == '최신':
    answer_t = '최근'
else:
    answer_t = 'tred error'

# 개체명
# movie = lsts[4][0]
# if len(movie) > 0:
    
# actor = lsts[4][1]
# genre = lsts[4][2]
# nation = lsts[4][3]
# director = lsts[4][4]
# date = lsts[4][5]
# rating = lsts[4][6]

In [65]:
answer = f"{answer_t} {answer_e} '{recomand_movie}'{answer_i}"
print(answer)

인기 있는 슬픈 영화인 '미션임파서블:고스트프로토콜'을(를) 추천드려요. :)
